In [2]:
# --------------------------------------------------------------
# Customer Support Ticket Classification System Test 2
# --------------------------------------------------------------

from pydantic_model import *
from tickets import *
from ollama_connect import *
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)  # or a large number instead of None

MODELNAME = "llama3.1:8b"
rows=[]
#MODELNAME = "gemma3:12b"

# --------------------------------------------------------------
# Step 5: Optimize your prompts and experiment
# --------------------------------------------------------------
# To optimize:
# 1. Refine the system message to provide more context about your business
# 2. Experiment with different models (e.g., gpt-3.5-turbo vs gpt-4)
# 3. Fine-tune the model on your specific ticket data if available
# 4. Adjust the TicketClassification model based on business needs

SYSTEM_PROMPT = """
You are an AI assistant for a large e-commerce platform's customer support team. 
Your role is to analyze incoming customer support tickets and provide structured information to help our team respond quickly and effectively.

Business Context:
- We handle thousands of tickets daily across various categories (orders, accounts, products, technical issues, billing).
- Quick and accurate classification is crucial for customer satisfaction and operational efficiency.
- We prioritize based on urgency and customer sentiment.

Your tasks:
1. Categorize the ticket into the most appropriate category.
2. Assess the urgency of the issue (low, medium, high, critical).
3. Determine the customer's sentiment.
4. Determine the amount of Personally identifiable information (PII) in the ticket.
5. Categorize the ticket into the most appropriate category for the type of Personally identifiable information (PII).
6. Extract key information that would be helpful for our support team.
7. Suggest an initial action for handling the ticket.
8. Provide a confidence score for your classification.

Remember:
- Be objective and base your analysis solely on the information provided in the ticket.
- If you're unsure about any aspect, reflect that in your confidence score.
- For 'key_information', extract specific details like order numbers, product names, or account issues.
- The 'suggested_action' should be a brief, actionable step for our support team.

Analyze the following customer support ticket and provide the requested information in the specified format.
"""

def classify_ticket(ticket_text: str) -> TicketClassification:
    response = client_instructor.chat.completions.create(
        model=MODELNAME,
        response_model=TicketClassification,
        temperature=0.0,
        max_retries=5,
        seed=633242,
        max_tokens=16384,
        top_p=0.2,
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT,
            },
            {"role": "user", "content": ticket_text}
        ]
    )
    return response

for ticket in tickets:
    result = classify_ticket(ticket)
    data = result.model_dump()
    data['raw_ticket'] = ticket.strip()
    rows.append(data)


df = pd.DataFrame(rows)

# Optional styling
styled_df = df.style.set_properties(**{
    'text-align': 'left',
    'max-width': '300px',
    'white-space': 'normal'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('padding', '6px')]},
])
